## Creating 4LAC for AGN Pop - version 3

In [1]:
from astropy.io import fits
import numpy as np
from astropy.table import QTable, Table
import astropy.units as u
from astropy.io import ascii

First set the paths of the catalog files. Note here I'm using the DR2 catalogs:

In [2]:
path_4fgl_dr3_catalog = 'input_catalogs/gll_psc_v31.fit'
path_4lac_low_gal_lat = 'input_catalogs/table-4LAC-DR3-l.fits'
path_4lac_high_gal_lat = 'input_catalogs/table-4LAC-DR3-h.fits'
# Paolo Goldoni's catalog (revised 4LAC redshifts) version 2: https://zenodo.org/record/5512660#.YVcoKHuxXRY 
path_goldoni_catalog = 'input_catalogs/4LAC_newz_AGNPop_Oct2023.fits'

In [3]:
def read_4fgl_dr3():
    hdulist = fits.open(path_4fgl_dr3_catalog)
    ptSrcCat = hdulist[1].data
    for i, name in enumerate(ptSrcCat['Source_Name']):
        name = name.replace("4FGL ", "")
        name = name.replace(" ", "")
        if name.endswith('c'):
            name = name.replace('c', '')
        ptSrcCat['Source_Name'][i] = name
    return ptSrcCat

def read_4lac_catalog_names(filename):
    hdulist = fits.open(filename)
    ptSrcCat = hdulist[1].data
    names = ptSrcCat['Source_Name']
    sed_class = ptSrcCat['SED_class']
    srcclass = ptSrcCat['CLASS']
    redshifts = ptSrcCat['Redshift']
    for i, name in enumerate(names):
        name = name.replace("4FGL ", "")
        name = name.replace(" ", "")
        names[i] = name.replace('c', '')
    return QTable([names, redshifts, srcclass, sed_class], names=('Source_Name', 'Redshift', 'CLASS', 'SED_class'))

def read_goldoni_catalog():
    hdulist = fits.open(path_goldoni_catalog)
    goldoni = hdulist[1].data
    names = goldoni['4FGL name']
    for i, name in enumerate(names):
        names[i] = name.replace("4FGL ","")
        if "J0947.1-25" in name:
            names[i] = "J0947.1-2541"
    goldoni['4FGL name'] = names
    return goldoni

def merge_4lac_and_4fgl():
    four_fgl = read_4fgl_dr3()
    four_lac_l = read_4lac_catalog_names(path_4lac_low_gal_lat)
    four_lac_h = read_4lac_catalog_names(path_4lac_high_gal_lat)
    print("Length of the 4fgl catalog: {}".format(len(four_fgl)))
    print("Length of the 4lac catalog: {}".format(len(four_lac_l) + len(four_lac_h)))

    mask_4lac = np.zeros(np.shape(four_fgl['Source_Name']), dtype=bool)
    # mask_4lac = mask_4lac*False
    for i, name in enumerate(four_fgl['Source_Name']):
        if name in four_lac_l['Source_Name']:
            mask_4lac[i] = True
        elif name in four_lac_h['Source_Name']:
            mask_4lac[i] = True
    lac_fgl_crosscatalog = QTable(four_fgl[mask_4lac])

    print("Total number of 4LAC entries found in the 4FGL: {}".format(len(lac_fgl_crosscatalog)))
    print("Total number of 4LAC entries: {}".format(len(four_lac_l)+len(four_lac_h)))
    
    # Create new column in the 4fgl
    lac_fgl_crosscatalog['Redshift'] = -1.
    redshifts_added = 0
    pos_redshifts_added = 0
    for i, name in enumerate(lac_fgl_crosscatalog['Source_Name']):
        if name in four_lac_l['Source_Name']:
            lac_fgl_crosscatalog['Redshift'][i] = four_lac_l['Redshift'][four_lac_l['Source_Name'] == name]
            redshifts_added += 1
            if four_lac_l['Redshift'][four_lac_l['Source_Name'] == name] > 0:
                pos_redshifts_added += 1
        elif name in four_lac_h['Source_Name']:
            lac_fgl_crosscatalog['Redshift'][i] = four_lac_h['Redshift'][four_lac_h['Source_Name'] == name]
            redshifts_added += 1
            if four_lac_h['Redshift'][four_lac_h['Source_Name'] == name] > 0:
                pos_redshifts_added += 1
        else:
            print("This source did not appear in neither of the 4LAC catalogs...")
    print("Added a total of {} redshifts. {} had positive values.".format(redshifts_added, pos_redshifts_added))
    return lac_fgl_crosscatalog

def add_class_and_sedclass_to_merged_4fgl_and_4lac():
    catalog = merge_4lac_and_4fgl()
    four_lac_l = read_4lac_catalog_names(path_4lac_low_gal_lat)
    four_lac_h = read_4lac_catalog_names(path_4lac_high_gal_lat)

    srcclass = []
    sed_class = []
    for source_name in catalog['Source_Name']:
        if source_name in four_lac_l['Source_Name']:
            srcclass.append(four_lac_l['CLASS'][four_lac_l['Source_Name'] == source_name][0])
            sed_class.append(four_lac_l['SED_class'][four_lac_l['Source_Name'] == source_name][0])
        elif source_name in four_lac_h['Source_Name']:
            srcclass.append(four_lac_h['CLASS'][four_lac_h['Source_Name'] == source_name][0])
            sed_class.append(four_lac_h['SED_class'][four_lac_h['Source_Name'] == source_name][0])
    catalog['CLASS'] = srcclass
    catalog['SED_class'] = sed_class
    return catalog
    
def goldoni_revised_4lac():
    catalog = add_class_and_sedclass_to_merged_4fgl_and_4lac()
    goldoni = read_goldoni_catalog()
    # Create some variables to store statistics:
    valid_redshifts = 0
    removed_valid_redshift = 0
    added_valid_redshift = 0
    updated_value = 0
    for i, name in enumerate(catalog['Source_Name']):
        if name in goldoni['4FGL name']:
#             goldoni_redshift = goldoni['Redshift'][goldoni['4FGL name'] == name][0]
            goldoni_redshift = goldoni['Redshift_corr'][goldoni['4FGL name'] == name][0]
            #if "{:2.3f}".format(catalog['Redshift'][i]) == "{:2.6f}".format(goldoni_redshift):
            #print(goldoni_redshift,catalog['Redshift'][i])
            if np.abs(goldoni_redshift-catalog['Redshift'][i])<1e-3:
            #if "{:2.3f}".format(catalog['Redshift'][i]) == "{:2.3f}".format(goldoni_redshift):
                valid_redshifts += 1
            elif catalog['Redshift'][i] > 0. and goldoni_redshift > 0.:
                updated_value += 1
            elif catalog['Redshift'][i] < 0. and goldoni_redshift > 0.:
                added_valid_redshift += 1
            elif catalog['Redshift'][i] > 0. and goldoni_redshift < 0.:
                removed_valid_redshift += 1
            catalog['Redshift'][i] = "{:2.6f}".format(goldoni_redshift)
        else:
            catalog['Redshift'][i] = "{:2.6f}".format(catalog['Redshift'][i])
#         else:
#             print("{} in P. Goldoni catalog, but not within 4LAC.".format(name))
    print(" -- From Paolo's catalog -- ")
    print("A total of {} redshifts were correct within 4LAC".format(valid_redshifts))
    print("Removed a total of {} redshifts from 4LAC".format(removed_valid_redshift))
    print("Added a total of {} redshifts to 4LAC".format(added_valid_redshift))
    print("Updated a total of {} redshifts of 4LAC".format(updated_value))
    return catalog

def convert_PLSuperExpCutoff_entries_to_LogParabola(catalog):
    for i, entry in enumerate(catalog):
        if entry['SpectrumType'] == 'PLSuperExpCutoff ':
            catalog['SpectrumType'][i] = 'LogParabola      '
    return catalog
    

def create_agn_pop_shared_4lac_catalog():
    catalog = goldoni_revised_4lac()
    keep_columns = ['Source_Name', 'RAJ2000', 'DEJ2000', 'Redshift', 'SpectrumType', 'Pivot_Energy', 
                    'PL_Flux_Density', 'PL_Index', 'LP_Flux_Density', 'LP_Index', 'LP_beta', 'CLASS', 'SED_class', 
                    'Variability_Index', 'Frac_Variability', 'Unc_Frac_Variability', 'Flux1000', 'Unc_Flux1000']
#                     'Flux_History', 'Unc_Flux_History']
    new_catalog = QTable()
    for column in keep_columns:
        new_catalog[column] = catalog[column]
    final_catalog = convert_PLSuperExpCutoff_entries_to_LogParabola(catalog[keep_columns])
    return final_catalog


Generate the catalog and store it in a couple of different formats:

In [4]:
agn_pop_catalog = create_agn_pop_shared_4lac_catalog()
ascii.write(agn_pop_catalog, 'resulting_catalogs/agn_pop_4lac_dr3.dat', overwrite=True)  
agn_pop_catalog.write('resulting_catalogs/agn_pop_4lac_dr3.ecsv', overwrite=True)  
agn_pop_catalog.write('resulting_catalogs/agn_pop_4lac_dr3.fits', overwrite=True)  

Length of the 4fgl catalog: 6659
Length of the 4lac catalog: 3814
Total number of 4LAC entries found in the 4FGL: 3814
Total number of 4LAC entries: 3814
Added a total of 3814 redshifts. 1874 had positive values.
 -- From Paolo's catalog -- 
A total of 204 redshifts were correct within 4LAC
Removed a total of 94 redshifts from 4LAC
Added a total of 31 redshifts to 4LAC
Updated a total of 29 redshifts of 4LAC


Only selecting those sources with a valid redshift:

In [5]:
sources_with_redshift = agn_pop_catalog[agn_pop_catalog['Redshift'] >= 0.]
ascii.write(sources_with_redshift, 'resulting_catalogs/agn_pop_4lac_dr3_with_z.dat', overwrite=True)  
sources_with_redshift.write('resulting_catalogs/agn_pop_4lac_dr3_with_z.ecsv', overwrite=True)  
sources_with_redshift.write('resulting_catalogs/agn_pop_4lac_dr3_with_z.fits', overwrite=True)  

In [6]:
sources_with_redshift.sort("PL_Flux_Density")

In [7]:
len(sources_with_redshift)

1811

In [8]:
sources_with_redshift

Source_Name,RAJ2000,DEJ2000,Redshift,SpectrumType,Pivot_Energy,PL_Flux_Density,PL_Index,LP_Flux_Density,LP_Index,LP_beta,CLASS,SED_class,Variability_Index,Frac_Variability,Unc_Frac_Variability,Flux1000,Unc_Flux1000
str18,float32,float32,float64,str17,float32,float32,float32,float32,float32,float32,str5,str3,float32,float32,float32,float32,float32
J2049.7-0036,312.4456,-0.616,0.25,PowerLaw,27414.951,1.5025078e-16,1.5459844,1.1948478e-16,1.5027565,-0.074055366,bll,,11.132641,0.0,10.0,4.2278694e-11,2.8191987e-11
J0854.0+2753,133.5155,27.884,0.494,PowerLaw,21657.176,2.0061331e-16,1.4452856,3.4816347e-16,0.44803974,0.8518391,bll,ISP,15.999075,0.0,10.0,3.343894e-11,1.6875893e-11
J0843.1+5034,130.7877,50.5741,0.439228,PowerLaw,16281.117,2.5256283e-16,1.5676596,4.2920565e-16,0.41477,0.92254114,bll,ISP,12.82761,0.0,10.0,3.271807e-11,1.651096e-11
J0350.0+0640,57.5043,6.6754,0.273,PowerLaw,23084.377,3.066681e-16,1.4946463,4.2683192e-16,1.3619263,0.19991502,bcu,,12.954952,0.0,10.0,6.068869e-11,2.7381084e-11
J0331.8-7040,52.969,-70.6697,0.277,LogParabola,19173.797,3.31714e-16,1.6244115,6.9445145e-16,0.9360819,0.90198624,bll,HSP,14.705848,0.0,10.0,2.450411e-11,7.535786e-12
J1130.5-3137,172.6499,-31.6219,0.150728,PowerLaw,20277.48,3.3305304e-16,1.5072083,3.4974859e-16,1.4859275,0.019613769,bll,HSP,12.132727,0.0,10.0,5.534836e-11,2.3528918e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J1522.1+3144,230.5454,31.7395,1.489,LogParabola,456.00125,8.9110004e-11,2.402245,9.728963e-11,2.3342924,0.07812083,fsrq,LSP,6841.0913,0.6339789,0.13539362,9.5542925e-09,1.3019524e-10
J1504.4+1029,226.1033,10.4978,1.839,LogParabola,518.219,1.0277302e-10,2.246488,1.13776655e-10,2.1219308,0.10057087,FSRQ,LSP,13923.587,0.8978677,0.19155036,1.9457499e-08,2.0496335e-10


In [9]:
sources_with_redshift[sources_with_redshift['SED_class'] == 'ISP']


Source_Name,RAJ2000,DEJ2000,Redshift,SpectrumType,Pivot_Energy,PL_Flux_Density,PL_Index,LP_Flux_Density,LP_Index,LP_beta,CLASS,SED_class,Variability_Index,Frac_Variability,Unc_Frac_Variability,Flux1000,Unc_Flux1000
str18,float32,float32,float64,str17,float32,float32,float32,float32,float32,float32,str5,str3,float32,float32,float32,float32,float32
J0854.0+2753,133.5155,27.884,0.494,PowerLaw,21657.176,2.0061331e-16,1.4452856,3.4816347e-16,0.44803974,0.8518391,bll,ISP,15.999075,0.0,10.0,3.343894e-11,1.6875893e-11
J0843.1+5034,130.7877,50.5741,0.439228,PowerLaw,16281.117,2.5256283e-16,1.5676596,4.2920565e-16,0.41477,0.92254114,bll,ISP,12.82761,0.0,10.0,3.271807e-11,1.651096e-11
J1403.4+4319,210.8684,43.3225,0.493249,PowerLaw,15546.501,3.3653146e-16,1.4860786,4.693328e-16,1.4917786,0.1527251,bll,ISP,14.394237,0.0,10.0,3.6494988e-11,1.4021048e-11
J1424.5+3705,216.1447,37.0954,0.289567,PowerLaw,12655.435,5.086558e-16,1.587459,6.457751e-16,1.4586804,0.13744408,bll,ISP,5.3397517,0.0,10.0,4.5419876e-11,1.7910081e-11
J1449.5+2746,222.3956,27.7686,0.030588,PowerLaw,14613.923,5.4574587e-16,1.4604453,8.3478845e-16,1.3663455,0.1986019,rdg,ISP,12.727665,0.0,10.0,5.2408255e-11,1.6213687e-11
J1443.6+2515,220.9028,25.2631,0.529022,PowerLaw,17298.791,5.5116354e-16,1.7164963,7.4830194e-16,1.7135084,0.19640021,bll,ISP,21.832237,0.0,10.0,9.8816226e-11,4.4633863e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J0050.7-0929,12.6753,-9.4936,0.635,LogParabola,959.8714,4.066104e-12,2.009811,4.3360424e-12,1.919361,0.05795893,BLL,ISP,173.04103,0.2895267,0.065945946,3.8891583e-09,1.01961675e-10
J1132.7+0034,173.1961,0.5737,0.678,LogParabola,786.52136,4.2410016e-12,2.129867,4.3823226e-12,2.0753903,0.03346603,bll,ISP,26.02259,0.11830224,0.04254161,2.3106457e-09,8.148408e-11


Now let's select those sources without a valid redshift:

In [10]:
sources_without_redshift = agn_pop_catalog[agn_pop_catalog['Redshift'] < 0.]
ascii.write(sources_without_redshift, 'resulting_catalogs/agn_pop_4lac_dr3_without_z.dat', overwrite=True)  
sources_without_redshift.write('resulting_catalogs/agn_pop_4lac_dr3_without_z.ecsv', overwrite=True)  
sources_without_redshift.write('resulting_catalogs/agn_pop_4lac_dr3_without_z.fits', overwrite=True)  

In [11]:
len(sources_without_redshift)

2003

Make sure that the two catalogs have the same sources as the 4LAC (3814, see above):

In [12]:
len(sources_without_redshift) + len(sources_with_redshift)

3814

In [13]:
fh = fits.open(path_4lac_high_gal_lat)
th = Table(fh[1].data)

In [14]:
agn_pop_catalog

Source_Name,RAJ2000,DEJ2000,Redshift,SpectrumType,Pivot_Energy,PL_Flux_Density,PL_Index,LP_Flux_Density,LP_Index,LP_beta,CLASS,SED_class,Variability_Index,Frac_Variability,Unc_Frac_Variability,Flux1000,Unc_Flux1000
str18,float32,float32,float64,str17,float32,float32,float32,float32,float32,float32,str5,str3,float32,float32,float32,float32,float32
J0001.2+4741,0.3126,47.6859,-inf,PowerLaw,2420.5752,2.1560312e-14,2.2716956,2.1965499e-14,2.2540812,0.012156134,bcu,ISP,25.313953,0.67588246,0.31285,1.2597956e-10,2.8771974e-11
J0001.2-0747,0.3151,-7.7971,-inf,PowerLaw,1721.6039,2.656894e-13,2.1166923,2.842831e-13,2.0789266,0.051182024,bll,LSP,46.780693,0.40656525,0.11186972,7.471219e-10,5.041939e-11
J0001.4-0010,0.3717,-0.1699,0.461516,PowerLaw,4307.0825,6.067356e-15,1.9391596,6.4888174e-15,1.6612232,0.13243835,bll,LSP,9.272764,0.0,10.0,1.0822458e-10,2.9144395e-11
J0001.5+2113,0.3815,21.2183,1.106,LogParabola,329.77374,4.683598e-11,2.65406,5.3715615e-11,2.5141594,0.15931939,fsrq,ISP,1910.9358,0.9961379,0.21347848,1.3473536e-09,6.172809e-11
J0001.6-4156,0.4165,-41.9425,-inf,PowerLaw,4047.03,1.8815414e-14,1.7751755,2.131597e-14,1.6938646,0.07275369,bcu,HSP,26.393343,0.49097705,0.17578249,2.8218167e-10,3.0908824e-11
J0001.8-2153,0.4647,-21.8865,-inf,PowerLaw,4429.934,4.94271e-15,1.8766632,7.494641e-15,1.7166204,0.40580785,bcu,LSP,24.557972,0.90285075,0.38010666,9.04753e-11,2.4425273e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J2359.1+1719,359.7756,17.3225,-inf,PowerLaw,2498.8909,2.3101153e-14,2.1126761,2.739969e-14,1.9953225,0.11684519,bcu,ISP,10.333145,0.0,10.0,1.4291587e-10,2.8333176e-11
J2359.2-3134,359.8167,-31.5832,0.99,PowerLaw,633.0502,9.404723e-13,2.6442769,9.687202e-13,2.6346161,0.023825364,fsrq,LSP,82.691574,1.0123546,0.2485515,1.7072874e-10,2.7292819e-11


In [15]:
print('List of column names:\n', agn_pop_catalog.colnames)  # list of column names
print('\nNumber of table rows:', len(agn_pop_catalog))      # number of table rows

List of column names:
 ['Source_Name', 'RAJ2000', 'DEJ2000', 'Redshift', 'SpectrumType', 'Pivot_Energy', 'PL_Flux_Density', 'PL_Index', 'LP_Flux_Density', 'LP_Index', 'LP_beta', 'CLASS', 'SED_class', 'Variability_Index', 'Frac_Variability', 'Unc_Frac_Variability', 'Flux1000', 'Unc_Flux1000']

Number of table rows: 3814
